<a href="https://colab.research.google.com/github/yf591/llm-toolkit/blob/main/LLM_SFT_%26_RAG_GradioUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tning と LangChainを使用したRAGの構築

**LangChain関連ライブラリは主に以下の役割を担っています**

*   **ドキュメントの読み込みと前処理 (RAGの情報源準備)**: PDFやテキストを読み込み、検索に適した形に分割する。
*   **テキストのベクトル化 (埋め込み生成)**: テキスト情報を数値ベクトルに変換する。
*   **ベクトルストアの構築と利用 (高速検索)**: ベクトル化された情報を格納し、質問に類似した情報を効率的に検索する。
*   **LLMとの連携**: Hugging FaceのモデルをLangChainの枠組みで扱えるようにする。
*   **プロンプト管理**: LLMへの指示を柔軟に構築する。
*   **RAGチェーンの構築 (推論フローの自動化)**: 情報検索と回答生成の一連の処理を簡単にまとめる。

これによって、RAGシステムの構築と、ファインチューニング済みモデルとの連携が効率的に行えるようになっています。

このプロジェクトで採用した「**RAGを用いたLLMファインチューニング**」というアプローチでは例として以下のように学習を進めています。

1.  **ベースモデル**
    *   `tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.1` という、既に広い知識で事前学習され、指示応答能力も持つモデルが出発点としています。

2.  **ファインチューニング用データの作成プロセス (`prepare_training_data` 関数内)**
    *   **指示データ (ichikaraデータセット)**: まず、`ichikara-instruction-003-001-1.json` から「指示（質問）」と「期待される回答の元（出力）」のペアを取得します。
    *   **RAGによるコンテキスト検索**: 各「指示（質問）」に対して、**"Reinforcement Learning: An Introduction" のPDFから構築したベクトルストア（FAISSインデックス）を使って、関連性の高いテキストチャンク（これがRAGコンテキスト）を検索・取得します。**
    *   **学習サンプルの構築**: 取得した「RAGコンテキスト」と、元の「指示（質問）」、そして `ichikara` データセットの「期待される回答」を組み合わせて、以下のような形式の1つの学習サンプル（プロンプト）を作成します。
        ```
        ### 指示:
        以下のコンテキスト情報を使用して、質問に対する回答を生成してください。
        コンテキスト情報に含まれる事実のみを使用し、含まれていない情報は推測しないでください。

        ### コンテキスト:
        { "Reinforcement Learning: An Introduction" から検索された関連テキストの一部 }

        ### 質問:
        { ichikaraデータセットからの指示/質問 }

        ### 回答:
        { ichikaraデータセットからの期待される出力 }
        ```
    *   このプロセスを `ichikara` データセットの全ての項目に対して行い、ファインチューニング用のデータセット全体を構築します。

3.  **ファインチューニングの実行 (`train_model` 関数内)**
    *   ベースモデルに対して、上記で作成した「RAGコンテキストを含むプロンプト」を入力として、「期待される回答」を教師データとして学習（ファインチューニング）を行います。
    *   この学習によって、モデルは**「与えられたコンテキスト（この場合は強化学習の教科書からの抜粋）を理解し、それに基づいて質問に答える」という能力を獲得・向上させようと試みています。**

**つまり、**

*   ベースモデル (`Llama-3.1-Swallow-8B-Instruct-v0.1`) に対して、
*   `ichikara` データセットの各指示/質問と、
*   それに対応して "Reinforcement Learning: An Introduction" からRAGで検索された**一部の関連テキスト（コンテキスト）**
*   これらがセットになったものを**学習データ（ファインチューニング用のデータ）**として使用しています。
*   ファインチューニングモデル（ "Reinforcement Learning: An Introduction" ＋ ichikaraデータセットの知識を持つ）」という部分について、モデルが持つのは「知識そのもの」というよりは、「"Reinforcement Learning: An Introduction" からの情報を参照して、ichikara データセットのスタイルで応答する能力・パターン」 と言った方がより正確かもしれません。

**重要なポイント:**

*   "Reinforcement Learning: An Introduction" の**全部**が一度に学習データとして使われるわけではありません。各 `ichikara`データセット の質問に対して、その質問に**関連する部分だけ**がRAGによって都度抽出され、コンテキストとして学習サンプルに組み込まれます。
*   ファインチューニングの目的は、モデルにRAGで提供される専門的なコンテキストを効果的に利用して、より質の高い回答を生成する能力を植え付けることです。

## このColab Notebookの使い方

1.  **セットアップ**
    *   Google Colabのランタイムタイプを「GPU」 (A100など高性能なもの) に設定します。
    *   セル1から順番に実行していきます。
    *   **セル3**で、`DATA_PATH` (学習用JSON) と `DOCS_PATH` (RAG用ドキュメント/ディレクトリ) を、Colabにアップロードした実際のファイルのパスに正しく設定してください。
2.  **ドキュメント処理 (セル7)**
    *   `DOCS_PATH` に指定したドキュメントを処理し、ベクトルストア (`faiss_index`) を作成します。
3.  **トレーニングデータ準備 (セル9)**
    *   `DATA_PATH` のJSONと、セル7で作成したベクトルストアを使って、Fine-Tunig用のデータセットを作成します。
4.  **ファインチューニング**
    *   **セル13**を実行して、デフォルトパラメータでモデルをFine-Tunigします。モデルは `OUTPUT_DIR` に保存されます。
    *   （オプション）より良い性能を目指す場合は、**セル14**の `DO_HYPERPARAMETER_TUNING = True` に設定して実行し、ハイパーパラメータ探索を行います。ただし、これにはかなり時間がかかるので注意してください。
5.  **推論 (セル16)**
    *   Fine-Tunigが完了したら、セル16を実行します。
    *   これにより、Fine-Tunig済みモデルとRAGシステムがロードされ、Gradioのチャットインターフェースが起動します。
    *   表示されたUIのテキストボックスに質問を入力して、チャットボットと対話ができます。

**注意点**

*   **メモリ**: ここで使用しているLlama3 8Bモデルでも、特にFine-Tunig中は多くのGPUメモリを消費します。Colab Pro A100であれば動作する可能性は高いですが、バッチサイズやシーケンス長などのパラメータ調整が必要になる場合があります。
*   **時間**: ドキュメント処理、特にFine-Tunigには時間がかかります。
*   **ファイルパス**: Colabのファイルシステム (`/content/`) 上のパスを正しく指定することが重要です。MyDriveをマウントした場合も然り。
*   **エラー対処**: 作成時点では最後まで実行できましたが、システム等のアップデートなどで各セルでエラーが発生した場合、そのセルのエラーメッセージを確認し、必要な修正を行ってから再実行してください。前のセルの結果に依存している場合があるので、関連するセルも確認が必要です。
*   **Gradio**: Colab上でGradio UIを実行すると、セルの出力としてUIが表示されそこでチャットができるようになります。

## 事前準備

In [1]:
# GPUメモリの確認
!nvidia-smi

Sat May 10 13:18:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive, output

# Google Driveをマウント（必要に応じて）
drive.mount('/content/drive')

Hugging Faceへのログインは以下の方法1～方法3のいずれかの方法でおこなうこと

In [ ]:
# Hugging Face ログイン方法1

from huggingface_hub import login

# Hugging Faceにログイン（方法2のColabのシークレットを利用してもよい。これはお好みで。）
login()  # トークンの入力を求められます

In [18]:
# Hugging Face ログイン方法2

from huggingface_hub import login
from google.colab import userdata # Google Colabのuserdataモジュールをインポート

# HuggingFaceアカウントにログイン
login(userdata.get('HF_TOKEN')) # Colabのシークレットキーを使用（Hugging FaceのAPIトークンを設定しておく必要があります。）

In [ ]:
# Hugging Face ログイン方法3

# 上記1または2の方法ででHugging Faceにログインできない場合は、".env"ファイルを作成してログインする
# HUGGINGFACE_TOKEN="*******************************"と書いて保存した.envファイルを"/content/.env"に置く

!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
!huggingface-cli login --token $$HF_WRITE_TOKEN

## 1.環境設定とライブラリインストール

- --quiet の部分は、pip (Python Package Installer) に対して、インストール時の詳細な出力（ログ）を抑制するように指示するオプション

**インストールした主なLangChain関連ライブラリについて補足**

*   `langchain` (コアライブラリ)
*   `langchain-community` (コミュニティによるインテグレーション、モデル、ローダーなど)
*   `faiss-cpu` (FAISSベクトルストアのCPU版。LangChainからベクトルストアとして利用)
*   `pypdf` (LangChainの`PyPDFLoader`がPDF読み込みに内部で使用)

In [1]:
# 環境設定とライブラリインストール

# PyTorch (Colabのデフォルト、または必要ならバージョン指定)
# 今回はColabのデフォルトに任せる

# 主要なHugging Faceライブラリ
!pip install transformers datasets accelerate --quiet

# LangChain関連
!pip install langchain langchain-community faiss-cpu sentencepiece --quiet

# QLoRA関連
!pip install bitsandbytes peft --quiet

# その他ユーティリティ
!pip install torch --quiet # torchを明示的に再度入れることで依存関係を再確認させる場合があるが、通常は不要かも
!pip install ray[tune] --quiet
!pip install pandas numpy scikit-learn tqdm --quiet
!pip install pypdf --quiet
!pip install gradio --quiet
!pip install protobuf --quiet # 明示的に追加

# 最後に fsspec と gcsfs のバージョンを強制的に調整
# datasets 3.6.0 (仮にこのバージョンがインストールされたとする) は fsspec[http]<=2025.3.0,>=2023.1.0 を要求
# なので、fsspec は 2025.3.0 に固定する
print("Attempting to install fsspec==2025.3.0...")
!pip install fsspec==2025.3.0 --quiet
# 次に、fsspec==2025.3.0 と互換性のある gcsfs (例: 2025.3.0) を指定
# このインストールは、もし上位のバージョンが入ってしまっていたらダウングレードする効果がある
print("Attempting to install gcsfs==2025.3.0...")
!pip install gcsfs==2025.3.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2.ライブラリのインポート
このセルでは、インストールしたライブラリやPythonの標準モジュールをインポートします。

### 主なLangChain関連ライブラリについて補足
`from langchain_community.embeddings import HuggingFaceEmbeddings`
- Hugging Faceのモデルを使ってテキストの埋め込みベクトルを生成するためのクラス。RAGでドキュメントや質問をベクトル化するのに使います。

`from langchain_community.vectorstores import FAISS`
- FAISSベクトルストアを扱うためのクラス。ベクトル化されたドキュメントを格納し、類似度検索を可能にします。

`from langchain_community.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader`
- PDFファイルやテキストファイル、ディレクトリ内のドキュメントを読み込むためのクラス。

`from langchain.text_splitter import RecursiveCharacterTextSplitter`
- 長いドキュメントを検索に適した小さなチャンクに分割するためのクラス。

`from langchain.chains import RetrievalQA`
- RAG（Retrieval Augmented Generation）の処理フローを簡単に構築するためのチェーン。リトリーバー（情報検索）とLLM（言語モデルによる回答生成）を組み合わせます。

`from langchain_community.llms import HuggingFacePipeline`
- Hugging Faceの`pipeline`（このノートブックではファインチューニング済みモデルを使ったテキスト生成パイプライン）をLangChainのLLMインターフェースに適合させるためのクラス。

`from langchain.prompts import PromptTemplate`
- LLMに入力するプロンプトのテンプレートを定義・管理するためのクラス。

`from langchain_core.documents import Document`
- LangChainがドキュメントを扱う際の基本となるデータ構造。ダミーデータ作成時などに使用。

In [4]:
import os
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from typing import List, Dict, Any
from tqdm import tqdm
from datasets import Dataset, load_dataset, concatenate_datasets
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging as hf_logging, # transformers.logging
    pipeline
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline # 正しいインポートパス
from langchain.prompts import PromptTemplate
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import gradio as gr # Gradio UI用
import json # JSONファイルの読み込み用
from langchain_core.documents import Document # ダミーDocument用

## 3.基本設定とグローバル変数
ここでは、モデルのパス、ディレクトリ、学習パラメータなど、プロジェクト全体で使用する主要な設定値を定義しています。ご自分の環境や目的に合わせて値を調整してください。

---
⚠️ 注意

DATA_PATH と DOCS_PATH には、Colabにアップロードしたファイルの正しいパスを指定してください。
Colabの左側にあるファイルアイコンからファイルをアップロードし、パスを右クリックしてコピーできます。また、MyDriveをマウントしてそこから読み取るように設定してもOKです。

- 例: /content/ichikara-instruction-003-001-1.json
- 例: /content/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf

- (単一PDFの場合)
  - 例: /content/my_documents_folder/ (複数ドキュメントを格納したフォルダの場合)

In [5]:
# 基本設定とグローバル変数

# --- 基本モデル設定 ---
# ベースモデル
BASE_MODEL = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.1" #@param {type:"string"}

# 埋め込みモデル
EMBEDDING_MODEL = "intfloat/multilingual-e5-large"   #@param {type:"string"}


# --- パス設定 ---
# トレーニングデータのパス（例: Colabにアップロードしたファイルのパス）
DATA_PATH = "/content/ichikara-instruction-003-001-1.json" #@param {type:"string"}

# RAG用ドキュメントのパス (単一ファイルまたはディレクトリ)
DOCS_PATH = "/content/Reinforcement Learning An Introduction Second edition.pdf"  #@param {type:"string"}

# ファインチューニング済みモデルの保存先
OUTPUT_DIR = "/content/finetuned_model" #@param {type:"string"}

# FAISSインデックスの保存場所
FAISS_INDEX_PATH = "faiss_index" #@param {type:"string"}


# --- LoRA設定 ---
LORA_R = 8 #@param {type:"number"}
LORA_ALPHA = 16 #@param {type:"number"}
LORA_DROPOUT = 0.05 #@param {type:"number"}


# --- RAGドキュメント処理設定 ---
CHUNK_SIZE = 1000 #@param {type:"number"}
CHUNK_OVERLAP = 200 #@param {type:"number"}

# PDFごとに処理する最大ページ数 (メモリ対策)
MAX_PAGES_PER_PDF = 352 #@param {type:"number"}


# --- トレーニングパラメータ (デフォルト) ---
# これらの値は tune_hyperparameters で最適化されるか、直接 train_model 関数に渡されます
DEFAULT_PER_DEVICE_BATCH_SIZE = 2 #@param {type:"number"}
DEFAULT_GRADIENT_ACCUMULATION_STEPS = 8 #@param {type:"number"}
DEFAULT_LEARNING_RATE = 2e-4 #@param {type:"number"}
DEFAULT_LOGGING_INTERVAL = 10 #@param {type:"number"}
DEFAULT_EVALUATION_INTERVAL = 100 #@param {type:"number"}
DEFAULT_CHECKPOINT_INTERVAL = 100 #@param {type:"number"}

# エポック数 (データ量に応じて調整)
DEFAULT_NUM_TRAIN_EPOCHS = 1 #@param {type:"number"}
DEFAULT_LR_SCHEDULER_TYPE = "cosine" #@param {type:"string"}
DEFAULT_WARMUP_RATIO = 0.03 #@param {type:"number"}


# --- その他 ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

print(f"使用デバイス: {DEVICE}")
print(f"ベースモデル: {BASE_MODEL}")
print(f"埋め込みモデル: {EMBEDDING_MODEL}")
print(f"ファインチューニング済みモデル出力先: {OUTPUT_DIR}")

使用デバイス: cuda
ベースモデル: tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.1
埋め込みモデル: intfloat/multilingual-e5-large
ファインチューニング済みモデル出力先: /content/finetuned_model


## 4.シード設定とロギング設定

ここでは再現性のために乱数シードを設定しています。またHugging Face Transformersライブラリのロギングレベルを設定しています。

In [6]:
# シード設定
torch.manual_seed(SEED)
np.random.seed(SEED)

# ロギング設定
hf_logging.set_verbosity_info() # Hugging Face Transformersのログレベル

## 5.量子化の設定

ビット量子化（QLoRA）のためのBitsAndBytes設定を返す関数get_bnb_config を定義します。

In [7]:
# BitsAndBytesの設定 (4ビット量子化)
def get_bnb_config():
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

## 6.ドキュメント読み込みと処理

ここでは、RAGの情報源となるドキュメントを読み込んで、前処理（チャンク分割、ベクトル化）を行います。また、FAISSベクトルストアを作成・保存する関数 load_and_process_documents も定義しています。

### RAG・LangChain関連の補足
*   `PyPDFLoader`, `TextLoader`: `DOCS_PATH` からPDFやテキストファイルを読み込み、LangChainの `Document` オブジェクトのリストとして取得します。
*   `RecursiveCharacterTextSplitter`: 読み込んだ `Document` をチャンクに分割します。
*   `HuggingFaceEmbeddings`: 分割されたテキストチャンクを埋め込みベクトルに変換します。
*   `FAISS`: 埋め込みベクトル化されたチャンクを格納し、高速な類似度検索が可能なベクトルストアを構築・保存します。

In [8]:
# ドキュメント読み込みと処理関数 (load_and_process_documents)
def load_and_process_documents(docs_path, faiss_index_save_path, chunk_size, chunk_overlap, max_pages_per_pdf):
    """
    RAG用のドキュメントを読み込み、テキストチャンクに分割し、
    ベクトルストア (FAISS) を作成してローカルに保存します。
    """
    print("ドキュメント読み込みと処理を開始...")

    documents = []

    if os.path.isfile(docs_path):
        print(f"単一ファイル {docs_path} を処理します。")
        file_ext = os.path.splitext(docs_path)[1].lower()
        if file_ext == ".pdf":
            print(f"PDFファイル {docs_path} を読み込み中...")
            try:
                loader = PyPDFLoader(docs_path, extract_images=False)
                pages = loader.load_and_split()
                documents.extend(pages[:max_pages_per_pdf])
                print(f"{docs_path} から最初の {min(len(pages), max_pages_per_pdf)}/{len(pages)} ページを読み込みました。")
            except Exception as e:
                print(f"PDF読み込みエラー ({docs_path}): {e}")
        elif file_ext == ".txt":
            print(f"テキストファイル {docs_path} を読み込み中...")
            try:
                loader = TextLoader(docs_path, encoding='utf-8') # エンコーディング指定
                documents.extend(loader.load())
            except Exception as e:
                print(f"テキストファイル読み込みエラー ({docs_path}): {e}")
        else:
            print(f"サポートされていないファイル形式です: {docs_path}")

    elif os.path.isdir(docs_path):
        print(f"ディレクトリ {docs_path} 内のファイルを処理します。")
        # PDFファイル読み込み
        pdf_files = [f for f in os.listdir(docs_path) if f.lower().endswith('.pdf')]
        for pdf_filename in tqdm(pdf_files, desc="PDFドキュメント読み込み"):
            pdf_path_full = os.path.join(docs_path, pdf_filename)
            print(f"PDFファイル {pdf_path_full} を読み込み中...")
            try:
                loader = PyPDFLoader(pdf_path_full, extract_images=False)
                pages = loader.load_and_split()
                documents.extend(pages[:max_pages_per_pdf])
                print(f"{pdf_filename} から最初の {min(len(pages), max_pages_per_pdf)}/{len(pages)} ページを読み込みました。")
            except Exception as e:
                print(f"PDF読み込みエラー ({pdf_path_full}): {e}")

        # テキストファイル読み込み
        txt_files = [f for f in os.listdir(docs_path) if f.lower().endswith('.txt')]
        for txt_filename in tqdm(txt_files, desc="TXTドキュメント読み込み"):
            txt_path_full = os.path.join(docs_path, txt_filename)
            print(f"テキストファイル {txt_path_full} を読み込み中...")
            try:
                loader = TextLoader(txt_path_full, encoding='utf-8') # エンコーディング指定
                documents.extend(loader.load())
            except Exception as e:
                print(f"テキストファイル読み込みエラー ({txt_path_full}): {e}")
    else:
        print(f"指定されたDOCS_PATH '{docs_path}' は有効なファイルまたはディレクトリではありません。")

    if not documents:
        print("読み込むドキュメントが見つかりませんでした。RAGのコンテキストは空になります。")
        # ダミーのドキュメントやエラー処理が必要な場合がある
        # ここでは空のベクトルストアが作成されるのを許容する

    # テキスト分割
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    texts = text_splitter.split_documents(documents)
    print(f"{len(texts)}個のテキストチャンクを作成しました。")

    if not texts:
        print("テキストチャンクが作成されませんでした。ベクトルストアは空になる可能性があります。")
        # FAISS.from_documentsが空のリストでエラーになるのを避けるためのダミーデータ
        texts = [Document(page_content="No valid content found to create embeddings.")]
        print("警告: 有効なテキストチャンクが生成されなかったため、ダミーコンテンツでベクトルストアを初期化します。")

    # 埋め込みモデルの初期化
    print(f"埋め込みモデル {EMBEDDING_MODEL} をロード中...")
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={"device": DEVICE}
    )

    # ベクトルストアの作成と保存
    print("ベクトルストアを作成中...")
    try:
        vectorstore = FAISS.from_documents(texts, embeddings)
        vectorstore.save_local(faiss_index_save_path)
        print(f"ベクトルストアを {faiss_index_save_path} に作成し、保存しました。")
    except Exception as e:
        print(f"ベクトルストア作成エラー: {e}")
        print("ベクトルストアの作成に失敗しました。以降のRAG処理に影響が出る可能性があります。")
        return None # エラー時はNoneを返す

    return vectorstore

## 7.ドキュメント読み込みとベクトルストア作成の実行

ここでは、セル6で定義された `load_and_process_documents` 関数を呼び出して、DOCS_PATH からドキュメントを読み込み、実際にLangChainの機能を使ってベクトルストアを作成します。


---
⚠️ DOCS_PATH が正しく設定されていることを確認してください。

In [9]:
# ドキュメント読み込みとベクトルストア作成の実行

# DOCS_PATHにファイル/ディレクトリが存在するか確認
if not os.path.exists(DOCS_PATH):
    print(f"エラー: DOCS_PATH '{DOCS_PATH}' が見つかりません。")
    print("Colabにドキュメントファイルまたはディレクトリをアップロードし、セル3のDOCS_PATHを正しく設定してください。")
    # このセル以降の処理が依存するため、ここで処理を中断することも検討
    vectorstore = None
else:
    vectorstore = load_and_process_documents(
        docs_path=DOCS_PATH,
        faiss_index_save_path=FAISS_INDEX_PATH,
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        max_pages_per_pdf=MAX_PAGES_PER_PDF
    )

if vectorstore:
    print("ベクトルストアの準備が完了しました。")
else:
    print("警告: ベクトルストアが正常に作成されませんでした。")

ドキュメント読み込みと処理を開始...
単一ファイル /content/Reinforcement Learning An Introduction Second edition.pdf を処理します。
PDFファイル /content/Reinforcement Learning An Introduction Second edition.pdf を読み込み中...
/content/Reinforcement Learning An Introduction Second edition.pdf から最初の 352/353 ページを読み込みました。
1030個のテキストチャンクを作成しました。
埋め込みモデル intfloat/multilingual-e5-large をロード中...


<ipython-input-8-9291a6e549bd>:83: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--intfloat--multilingual-e5-large/snapshots/0dc5580a448e4284468b8909bae50fa925907bc5/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--intfloat--multilingual-e5-large/snapshots/0dc5580a448e4284468b8909bae50fa925907bc5/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--intfloat--multilingual-e5-large/snapshots/0dc5580a448e4284468b8909bae50fa925907bc5/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--intfloat--multilingual-e5-large/snapshots/0dc5580a448e4284468b8909bae50fa925907bc5/tokenizer_config.json
loading file chat_template.jinja from cache at None


config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

ベクトルストアを作成中...
ベクトルストアを faiss_index に作成し、保存しました。
ベクトルストアの準備が完了しました。


## 8.トレーニングデータの準備

ここでは、ファインチューニング用のデータセットを準備する関数 prepare_training_data を定義します。

DATA_PATH (例: ichikara-instruction-003-001-1.json) から指示と回答のペアを読み込み、各指示に対してRAG（上記7のセルで作成した vectorstore）で関連コンテキストを検索して、学習用のプロンプトを生成します。

### RAG・Langchain関連の補足
*   `vectorstore.as_retriever()`: セル7で作成したFAISSベクトルストアから、LangChainの `Retriever` インターフェースを取得します。これを使って、トレーニングデータの各質問に関連するコンテキスト情報を検索します。
*   `retriever.get_relevant_documents(question)`: 質問に基づいて関連ドキュメント（コンテキスト）を取得します。

In [10]:
# トレーニングデータ準備関数 (prepare_training_data)
def prepare_training_data(data_path, vectorstore, seed):
    """
    ファインチューニング用のデータセットを準備します。
    指定されたJSONデータから指示と回答を読み込み、RAGでコンテキストを付加します。
    """
    print("トレーニングデータ準備中...")

    raw_data_list = []

    if not vectorstore:
        print("警告: ベクトルストアが利用できません。RAGコンテキストなしで進めますが、品質に影響する可能性があります。")
        # vectorstoreがNoneの場合のダミーretriever (何も返さない)
        class DummyRetriever:
            def get_relevant_documents(self, query): return []
        retriever = DummyRetriever()
    else:
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # 関連情報を3件取得

    try:
        with open(data_path, 'r', encoding='utf-8') as f:
            instruction_data_json = json.load(f)

        # JSONがリスト形式であることを想定 (Hugging Face datasets.load_dataset("json", ...) が期待する形式)
        if not isinstance(instruction_data_json, list):
            # もしルートが辞書で、キー (例: "train") の下にリストがある場合は調整
            # 例: if isinstance(instruction_data_json, dict) and "train" in instruction_data_json:
            #         instruction_data_json = instruction_data_json["train"]
            # 今回のichikara-instructionの形式はリストなので、そのままで良いはず
            pass

        print(f"{data_path} から {len(instruction_data_json)} 件のデータを読み込みました。")

        for item in tqdm(instruction_data_json, desc="RAGコンテキスト検索とデータ整形"):
            question = item.get("text") # JSONの "text" を質問として使用
            ground_truth_answer = item.get("output") # JSONの "output" を正解回答として使用

            if question is None or ground_truth_answer is None:
                print(f"警告: 不完全なデータエントリをスキップします: {item}")
                continue

            # 質問に基づいて関連ドキュメントをベクトルストアから取得
            docs = retriever.get_relevant_documents(question)
            context = "\n\n".join([doc.page_content for doc in docs])

            raw_data_list.append({
                "question": question,
                "context": context,
                "answer": ground_truth_answer
            })

    except FileNotFoundError:
        print(f"エラー: トレーニングデータファイル {data_path} が見つかりません。")
        print("フォールバックとしてサンプルデータを生成します... (これはテスト用です)")
        # サンプルデータ (フォールバック)
        questions = [
            "このプロジェクトの目的は何ですか？", "主要な機能はどのようなものですか？",
            "どのようなユースケースがありますか？", "類似プロジェクトはありますか？",
            "技術的な課題は何ですか？"
        ]
        for question in questions:
            docs = retriever.get_relevant_documents(question)
            context = "\n\n".join([doc.page_content for doc in docs])
            answer = f"コンテキスト情報に基づく回答です。この回答はサンプルです。{context[:100]}..."
            raw_data_list.append({"question": question, "context": context, "answer": answer})
    except json.JSONDecodeError:
        print(f"エラー: {data_path} のJSON形式が正しくありません。")
        return None, None # エラー時はNoneを返す
    except Exception as e:
        print(f"トレーニングデータ読み込み/処理中に予期せぬエラーが発生しました: {e}")
        return None, None # エラー時はNoneを返す

    if not raw_data_list:
        print("エラー: 有効なトレーニングデータが生成されませんでした。")
        return None, None

    raw_data_hf_dataset = Dataset.from_list(raw_data_list)

    # RAGプロンプトテンプレートの作成
    def create_rag_prompt(sample):
        return f"""### 指示:
以下のコンテキスト情報を使用して、質問に対する回答を生成してください。
コンテキスト情報に含まれる事実のみを使用し、含まれていない情報は推測しないでください。

### コンテキスト:
{sample['context']}

### 質問:
{sample['question']}

### 回答:
{sample['answer']}
"""

    # データセットの変換
    try:
        processed_data = raw_data_hf_dataset.map(
            lambda sample: {"text": create_rag_prompt(sample)}, # ここを修正
            remove_columns=raw_data_hf_dataset.column_names
        )
    except Exception as e:
        print(f"データセットのmap処理中にエラー: {e}")
        print("raw_data_hf_dataset の内容:", raw_data_hf_dataset[0] if len(raw_data_hf_dataset) > 0 else "空")
        return None, None

    # トレーニングセットとバリデーションセットに分割
    if len(processed_data) < 10: # データが非常に少ない場合 (閾値は調整可能)
        print("警告: データセットのサンプル数が非常に少ないです。")
        if len(processed_data) < 2:
            print("サンプル数が2未満のため、トレーニング/検証セットの分割は行いません。すべてトレーニングデータとして使用します。")
            train_data = processed_data
            # 検証データがないとTrainerがエラーになる場合があるので、トレーニングデータをコピーする
            # または、evaluation_strategy="no" にするなどの対応が必要
            val_data = processed_data # 実際の学習では推奨されない
        else:
            # 非常に少ない場合は、test_sizeを固定値にするか、全量をtrainにする
            train_val_split = processed_data.train_test_split(test_size=0.1, seed=seed, shuffle=True) # shuffle=True を明示
            train_data = train_val_split["train"]
            val_data = train_val_split["test"]
    else:
        train_val_split = processed_data.train_test_split(test_size=0.1, seed=seed, shuffle=True)
        train_data = train_val_split["train"]
        val_data = train_val_split["test"]

    print(f"トレーニングサンプル数: {len(train_data)}")
    print(f"検証サンプル数: {len(val_data)}")

    return train_data, val_data

## 9.トレーニングデータ準備の実行

ここでは、実際に prepare_training_data 関数を呼び出して、トレーニングデータと検証データを作成します。

---

⚠️ DATA_PATH が正しく設定されていること、およびセル7で vectorstore が正常に作成されていることを確認してください。


In [11]:
# トレーニングデータ準備の実行

if not os.path.exists(DATA_PATH):
    print(f"エラー: DATA_PATH '{DATA_PATH}' が見つかりません。")
    print("Colabにトレーニングデータファイル (JSON) をアップロードし、セル3のDATA_PATHを正しく設定してください。")
    train_data, val_data = None, None
elif vectorstore is None and "DummyRetriever" not in str(prepare_training_data.__globals__): # セル8が正常に実行されたか
    print(f"エラー: ベクトルストアが初期化されていません。セル7を先に実行してください。")
    train_data, val_data = None, None
else:
    train_data, val_data = prepare_training_data(
        data_path=DATA_PATH,
        vectorstore=vectorstore, # セル7で作成されたもの
        seed=SEED
    )

if train_data and val_data:
    print("トレーニングデータと検証データの準備が完了しました。")
    print("トレーニングデータの最初のサンプル:", train_data[0] if len(train_data) > 0 else "データなし")
else:
    print("エラー: トレーニングデータまたは検証データの準備に失敗しました。ログを確認してください。")

トレーニングデータ準備中...
/content/ichikara-instruction-003-001-1.json から 1729 件のデータを読み込みました。


RAGコンテキスト検索とデータ整形:   0%|          | 0/1729 [00:00<?, ?it/s]<ipython-input-10-1fc8edc5bf81>:43: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)
RAGコンテキスト検索とデータ整形: 100%|██████████| 1729/1729 [00:41<00:00, 41.80it/s]


Map:   0%|          | 0/1729 [00:00<?, ? examples/s]

トレーニングサンプル数: 1556
検証サンプル数: 173
トレーニングデータと検証データの準備が完了しました。
トレーニングデータの最初のサンプル: {'text': '### 指示:\n以下のコンテキスト情報を使用して、質問に対する回答を生成してください。\nコンテキスト情報に含まれる事実のみを使用し、含まれていない情報は推測しないでください。\n\n### コンテキスト:\n166 CHAPTER 6. TEMPORAL-DIFFERENCE LEARNING\n\nthe driving home example and how it is addressed by TD and Monte Carlo\nmethods. Can you imagine a scenario in which a TD update would be better on\naverage than an Monte Carlo update? Give an example scenario—a description\nof past experience and a current state—in which you would expect the TD\nupdate to be better. Here’s a hint: Suppose you have lots of experience\ndriving home from work. Then you move to a new building and a new parking\nlot (but you still enter the highway at the same place). Now you are starting\nto learn predictions for the new building. Can you see why TD updates are\nlikely to be much better, at least initially, in this case? Might the same sort\nof thing happen in the original task?\nExercise 6.2 From Figure 6.6, it appears

## 10.モデルのロード

ここでは、ベースモデルをロードし、QLoRAのために4ビット量子化とLoRA設定を適用する関数 load_model_for_training を定義しています。

In [12]:
# モデルロード関数 (load_model_for_training)
def load_model_for_training(base_model_name, lora_r, lora_alpha, lora_dropout):
    """
    4ビット量子化とLoRA設定を適用してベースモデルをロードします。
    """
    print(f"ベースモデル {base_model_name} をトレーニング用にロード中...")

    bnb_config = get_bnb_config() # セル5で定義

    try:
        model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=bnb_config,
            device_map="auto", # 自動的にGPUに割り当て
            trust_remote_code=True
        )

        tokenizer = AutoTokenizer.from_pretrained(
            base_model_name,
            trust_remote_code=True
        )
        tokenizer.pad_token = tokenizer.eos_token # Llamaではeos_tokenをpad_tokenとして使うことが多い
        tokenizer.padding_side = "right" # 右パディング

        # モデルをkビットトレーニング用に準備
        model = prepare_model_for_kbit_training(model)

        # LoRAの設定
        lora_config = LoraConfig(
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"] # Llama3で一般的に対象とされるモジュール
        )

        model = get_peft_model(model, lora_config)

        model.print_trainable_parameters() # トレーニング可能なパラメータ数を表示

        print(f"モデル {base_model_name} のロードとLoRA設定が完了しました。")
        return model, tokenizer
    except Exception as e:
        print(f"モデルロード中にエラーが発生しました: {e}")
        return None, None

## 11.トレーニング関数の定義
ここでは、Hugging Face TransformersのTrainerの代わりに、PyTorchのカスタムトレーニングループを使用してモデルのファインチューニングを行う関数 train_model を定義しています。
これは、TrainingArguments の多くの設定を直接制御できるようにするためです。

In [13]:
# トレーニング関数 (train_model)

import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm # tqdm.auto を使うことでノートブック環境などにも対応
import os
import math # ステップ数計算で使用

# --- ヘルパー関数: オプティマイザとスケジューラの準備 ---
def _create_optimizer_and_scheduler(
    model_parameters,
    learning_rate,
    total_training_steps,
    warmup_steps,
    scheduler_name
):
    """オプティマイザと学習率スケジューラを作成して返します。"""
    optimizer = torch.optim.AdamW(model_parameters, lr=learning_rate)

    if scheduler_name == "cosine":
        # コサインアニーリングスケジューラ
        # T_maxはウォームアップ後の総ステップ数
        effective_t_max = total_training_steps - warmup_steps
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=max(1, effective_t_max) # T_maxは1以上である必要あり
        )
    elif scheduler_name == "linear":
        # 線形ウォームアップと線形減衰を行うスケジューラ
        def lr_lambda_linear(current_step):
            if current_step < warmup_steps:
                return float(current_step) / float(max(1, warmup_steps))
            return max(
                0.0,
                float(total_training_steps - current_step) / float(max(1, total_training_steps - warmup_steps))
            )
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda_linear)
    else: # デフォルトは定数スケジューラ (変更なし)
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda _: 1.0)

    return optimizer, scheduler

# --- ヘルパー関数: 1トレーニングバッチの処理 ---
def _process_train_batch(model, data_batch, target_device, accumulation_factor):
    """
    1つのトレーニングバッチを処理し、ロスを計算・返却します。
    勾配計算もここで行います。
    """
    input_ids = data_batch['input_ids'].to(target_device)
    attention_mask = data_batch['attention_mask'].to(target_device)
    # Causal Language Modelingでは、input_ids自体を教師ラベルとして使用
    labels = input_ids.clone()

    # モデルフォワードパス
    model_outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = model_outputs.loss

    # 勾配蓄積のためのロススケーリング
    scaled_loss = loss / accumulation_factor
    scaled_loss.backward() # スケールされたロスで勾配計算

    return loss.item() # スケーリング前のバッチロスを返す

# --- ヘルパー関数: モデル評価 ---
def _run_evaluation(model, evaluation_dataloader, target_device):
    """検証データセットでモデルを評価し、平均ロスを返します。"""
    model.eval() # モデルを評価モードに切り替え
    total_loss_eval = 0

    # プログレスバーの設定
    eval_bar = tqdm(evaluation_dataloader, desc="Model Evaluation", leave=False, dynamic_ncols=True)

    with torch.no_grad(): # 評価中は勾配計算を無効化
        for data_batch in eval_bar:
            input_ids = data_batch['input_ids'].to(target_device)
            attention_mask = data_batch['attention_mask'].to(target_device)
            labels = input_ids.clone()

            model_outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            batch_loss = model_outputs.loss
            total_loss_eval += batch_loss.item()

    average_eval_loss = total_loss_eval / len(evaluation_dataloader)
    model.train() # モデルをトレーニングモードに戻す
    return average_eval_loss

# --- メイントレーニング関数 (train_model の代替) ---
def train_model(
    model_to_train,
    tokenizer_for_model,
    training_dataset,
    validation_dataset,
    save_path_root,
    train_batch_size,
    gradient_accumulation_steps,
    max_epochs,
    base_learning_rate,
    lr_scheduler_name="cosine",
    warmup_ratio_of_total_steps=0.03,
    logging_interval_steps=10,
    evaluation_interval_steps=100,
    checkpoint_save_interval_steps=100,
    compute_device="cuda"
):
    """
    PyTorchカスタムループを使用してモデルのトレーニングを実行します (再構成版)。
    内部処理をヘルパー関数に分割し、変数名やコメントを変更しています。
    """
    print("モデルトレーニングを開始 (カスタムループ - 再構成版 v2)...")

    # --- データ準備フェーズ ---
    def _collate_and_tokenize_data(batch_samples):
        # "text" フィールドに整形済みプロンプトが含まれていることを想定
        return tokenizer_for_model(
            batch_samples["text"],
            padding="max_length", # 最大長までパディング
            truncation=True,      # 最大長を超える場合は切り捨て
            max_length=512       # モデルの最大コンテキスト長に合わせて調整
        )

    print("トレーニングデータをトークン化・フォーマット中...")
    processed_train_data = training_dataset.map(
        _collate_and_tokenize_data, batched=True, remove_columns=["text"]
    )
    processed_train_data.set_format("torch")

    print("検証データをトークン化・フォーマット中...")
    processed_val_data = validation_dataset.map(
        _collate_and_tokenize_data, batched=True, remove_columns=["text"]
    )
    processed_val_data.set_format("torch")

    # DataLoaderの作成
    train_dl = DataLoader(processed_train_data, batch_size=train_batch_size, shuffle=True)
    val_dl = DataLoader(processed_val_data, batch_size=train_batch_size) # 検証も同じバッチサイズ

    # --- トレーニング設定の計算 ---
    # 1エポックあたりのオプティマイザステップ数
    optimizer_steps_per_epoch = math.ceil(len(train_dl) / gradient_accumulation_steps)
    # 総オプティマイザステップ数 (トレーニング全体)
    total_optimizer_steps = optimizer_steps_per_epoch * max_epochs
    # ウォームアップステップ数
    num_warmup_optimizer_steps = int(total_optimizer_steps * warmup_ratio_of_total_steps)

    print(f"計画された総オプティマイザステップ数: {total_optimizer_steps}")
    print(f"うち、ウォームアップステップ数: {num_warmup_optimizer_steps}")

    # オプティマイザとスケジューラのインスタンス化
    optimizer, lr_scheduler = _create_optimizer_and_scheduler(
        model_to_train.parameters(),
        base_learning_rate,
        total_optimizer_steps,
        num_warmup_optimizer_steps,
        lr_scheduler_name
    )

    model_to_train.to(compute_device) # モデルを計算デバイスに移動

    # --- トレーニング状態変数の初期化 ---
    min_validation_loss_achieved = float('inf')
    optimizer_steps_completed = 0 # 実際にoptimizer.step()が実行された回数

    # --- メイントレーニングループ ---
    for current_epoch in range(int(max_epochs)):
        model_to_train.train() # エポック開始時にモデルをトレーニングモードに設定
        running_epoch_loss = 0.0 # このエポックの累積ロス

        # エポックごとのトレーニングプログレスバー
        epoch_train_bar = tqdm(
            enumerate(train_dl),
            total=len(train_dl),
            desc=f"Epoch {current_epoch + 1}/{int(max_epochs)} [Training]",
            dynamic_ncols=True # ターミナル幅に応じてバーの長さを調整
        )

        for batch_index, current_batch in epoch_train_bar:
            # 1バッチのトレーニング処理を実行
            batch_loss_value = _process_train_batch(
                model_to_train, current_batch, compute_device, gradient_accumulation_steps
            )
            running_epoch_loss += batch_loss_value

            # 勾配蓄積ステップ数に達したら、パラメータを更新
            if (batch_index + 1) % gradient_accumulation_steps == 0:
                optimizer.step()    # オプティマイザによるパラメータ更新
                lr_scheduler.step() # 学習率スケジューラによる更新
                optimizer.zero_grad() # 勾配をリセット
                optimizer_steps_completed += 1

                # --- 定期的なアクション (ロギング、評価、保存) ---
                # ロギング
                if optimizer_steps_completed % logging_interval_steps == 0:
                    current_learning_rate = optimizer.param_groups[0]['lr']
                    # 蓄積ステップ間の平均ロス（概算）
                    avg_loss_since_last_log = running_epoch_loss / (batch_index + 1) if batch_index > 0 else batch_loss_value
                    print(
                        f"Opt. Step: {optimizer_steps_completed}, LR: {current_learning_rate:.3e}, "
                        f"Avg Train Loss (epoch part): {avg_loss_since_last_log / gradient_accumulation_steps:.4f}"
                    )

                # 評価
                if evaluation_interval_steps > 0 and optimizer_steps_completed % evaluation_interval_steps == 0:
                    current_validation_loss = _run_evaluation(model_to_train, val_dl, compute_device)
                    print(f"Opt. Step {optimizer_steps_completed}: Validation Loss = {current_validation_loss:.4f}")

                    # 最良モデルの更新と保存
                    if current_validation_loss < min_validation_loss_achieved:
                        min_validation_loss_achieved = current_validation_loss
                        best_model_save_dir = os.path.join(save_path_root, "best_performing_model")
                        os.makedirs(best_model_save_dir, exist_ok=True)
                        model_to_train.save_pretrained(best_model_save_dir)
                        tokenizer_for_model.save_pretrained(best_model_save_dir)
                        print(
                            f"New best model checkpoint saved to {best_model_save_dir} "
                            f"(Validation Loss: {min_validation_loss_achieved:.4f})"
                        )

                # 定期的なチェックポイント保存
                if checkpoint_save_interval_steps > 0 and optimizer_steps_completed % checkpoint_save_interval_steps == 0:
                    chkpt_save_dir = os.path.join(save_path_root, f"checkpoint_opt_step_{optimizer_steps_completed}")
                    os.makedirs(chkpt_save_dir, exist_ok=True)
                    model_to_train.save_pretrained(chkpt_save_dir)
                    tokenizer_for_model.save_pretrained(chkpt_save_dir)
                    print(f"Checkpoint saved to {chkpt_save_dir} at optimizer step {optimizer_steps_completed}")

        # エポック終了時のサマリー
        avg_epoch_loss_display = running_epoch_loss / len(train_dl)
        print(f"End of Epoch {current_epoch + 1}: Average Training Loss = {avg_epoch_loss_display:.4f}")

    # --- トレーニング完了後の最終処理 ---
    final_model_save_dir = os.path.join(save_path_root, "final_trained_model")
    os.makedirs(final_model_save_dir, exist_ok=True)
    model_to_train.save_pretrained(final_model_save_dir)
    tokenizer_for_model.save_pretrained(final_model_save_dir)
    print(f"Training successfully completed. Final model saved to {final_model_save_dir}")

    return model_to_train, tokenizer_for_model

## 12.モデルのロードとトレーニングの実行 (チューニングなし)

ここでは、ハイパーパラメータチューニングを行わず、セル3で定義されたデフォルトのパラメータ、または手動で調整したパラメータを使用してモデルのロードとファインチューニングを実行します。

---
⚠️ セル9で train_data, val_data が正常に準備されていることを確認してください。

⚠️ セル13を実行する場合、このセルは実行しないでください

⚠️ このセルを実行する場合、セル14は実行しないでください（またはその逆）。

In [14]:
# モデルのロードとトレーニングの実行 (チューニングなし)

# このセルを実行する前に、train_dataとval_dataが準備されていることを確認
if 'train_data' not in globals() or train_data is None or \
   'val_data' not in globals() or val_data is None:
    print("エラー: train_data または val_data が定義されていません。セル9を先に実行してください。")
else:
    print("デフォルトパラメータでモデルのトレーニングを開始します...")
    # 1. モデルとトークナイザーのロード
    model, tokenizer = load_model_for_training(
        base_model_name=BASE_MODEL,
        lora_r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT
    )

    if model and tokenizer:
        # 2. トレーニングの実行
        trained_model, trained_tokenizer = train_model(
            model_to_train=model,
            tokenizer_for_model=tokenizer,
            training_dataset=train_data,
            validation_dataset=val_data,
            save_path_root=OUTPUT_DIR,
            train_batch_size=DEFAULT_PER_DEVICE_BATCH_SIZE,
            gradient_accumulation_steps=DEFAULT_GRADIENT_ACCUMULATION_STEPS,
            max_epochs=DEFAULT_NUM_TRAIN_EPOCHS,
            base_learning_rate=DEFAULT_LEARNING_RATE,
            lr_scheduler_name=DEFAULT_LR_SCHEDULER_TYPE,
            warmup_ratio_of_total_steps=DEFAULT_WARMUP_RATIO,
            logging_interval_steps=DEFAULT_LOGGING_INTERVAL,
            evaluation_interval_steps=DEFAULT_EVALUATION_INTERVAL,
            checkpoint_save_interval_steps=DEFAULT_CHECKPOINT_INTERVAL,
            compute_device=DEVICE
        )
        print(f"トレーニングが完了し、モデルは {OUTPUT_DIR} に保存されました。")
    else:
        print("モデルのロードに失敗したため、トレーニングを実行できませんでした。")

デフォルトパラメータでモデルのトレーニングを開始します...
ベースモデル tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.1 をトレーニング用にロード中...


config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--tokyotech-llm--Llama-3.1-Swallow-8B-Instruct-v0.1/snapshots/f82e2c268c583ca82f372bbc6825d19122e10bca/config.json
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--tokyotech-llm--Llama-3.1-Swallow-8B-Instruct-v0.1/snapshots/f82e2c268c583ca82f372bbc6825d19122e10bca/model.safetensors.index.json


...

Opt. Step: 10, LR: 1.947e-04, Avg Train Loss (epoch part): 0.2648
Opt. Step: 20, LR: 1.793e-04, Avg Train Loss (epoch part): 0.2308
Opt. Step: 30, LR: 1.556e-04, Avg Train Loss (epoch part): 0.2019
Opt. Step: 40, LR: 1.259e-04, Avg Train Loss (epoch part): 0.1847
Opt. Step: 50, LR: 9.346e-05, Avg Train Loss (epoch part): 0.1702
Opt. Step: 60, LR: 6.173e-05, Avg Train Loss (epoch part): 0.1578
Opt. Step: 70, LR: 3.407e-05, Avg Train Loss (epoch part): 0.1478
Opt. Step: 80, LR: 1.340e-05, Avg Train Loss (epoch part): 0.1397
Opt. Step: 90, LR: 1.921e-06, Avg Train Loss (epoch part): 0.1340


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--tokyotech-llm--Llama-3.1-Swallow-8B-Instruct-v0.1/snapshots/f82e2c268c583ca82f372bbc6825d19122e10bca/config.json
`rope_scaling`'s original_max_position_embeddings field must be less than max_position_embeddings, got 8192 and max_position_embeddings=8192
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "

End of Epoch 1: Average Training Loss = 1.0376


tokenizer config file saved in /content/finetuned_model/final_trained_model/tokenizer_config.json
Special tokens file saved in /content/finetuned_model/final_trained_model/special_tokens_map.json


Training successfully completed. Final model saved to /content/finetuned_model/final_trained_model
トレーニングが完了し、モデルは /content/finetuned_model に保存されました。


## 13.（オプション）ハイパーパラメータチューニング

ここでは、Ray Tuneを使用して最適なハイパーパラメータを探索する関数 tune_hyperparameters を定義しています。

---
⚠️  ハイパーパラメータチューニングは多くの計算リソースと時間を必要とします。小規模な試行から始めることをお勧めします。

⚠️  このセルを実行する場合、セル12の実行は不要です。

In [ ]:
# (オプション) ハイパーパラメータチューニング関数 (tune_hyperparameters)
def tune_hyperparameters(base_model_name, train_data, val_data, output_dir_root, device):
    """Ray Tuneを使用してハイパーパラメータを最適化します."""
    print("ハイパーパラメータチューニングを開始...")

    # Ray Tuneのトレーニング関数ラッパー
    def train_with_params_for_ray(config): # configはRay Tuneから渡される
        print(f"Tuning with config: {config}")

        # 各試行ごとにモデルをロード
        model, tokenizer = load_model_for_training(
            base_model_name=base_model_name,
            lora_r=config["lora_r"], # LoRAのRもチューニング対象にする場合
            lora_alpha=config["lora_r"] * 2, # 一般的にRの2倍
            lora_dropout=LORA_DROPOUT # 固定値またはチューニング対象
        )
        if model is None or tokenizer is None:
             ray.train.report({"val_loss": float('inf')}) # モデルロード失敗
             return

        # トレーニングの実行 (カスタムループ版を使う)
        # train_model関数は検証を行い、検証ロスを返すようには直接設計されていないため、
        # ここでは最後の検証ロスをRay Tuneにレポートする簡易的な方法を取るか、
        # train_modelを修正して定期的にval_lossをray.train.reportで報告するようにする。
        # 簡単のため、ここではトレーニング後の最終モデルで評価する想定 (あるいはTrainerを使う)

        # Trainerを使う場合の例 (元のコードに近い形)
        temp_output_dir = os.path.join(output_dir_root, f"tune_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{ray.train.get_context().get_trial_id()}")

        # トークン化
        def tokenize_function(examples):
            return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
        tokenized_train = train_data.map(tokenize_function, batched=True, remove_columns=["text"])
        tokenized_val = val_data.map(tokenize_function, batched=True, remove_columns=["text"])
        tokenized_train.set_format("torch")
        tokenized_val.set_format("torch")

        # TrainingArgumentsをRay Tuneのconfigで設定
        training_args = TrainingArguments(
            output_dir=temp_output_dir,
            per_device_train_batch_size=config["batch_size"],
            per_device_eval_batch_size=config["batch_size"], # 評価も同じバッチサイズ
            gradient_accumulation_steps=config["gradient_accumulation_steps"],
            learning_rate=config["learning_rate"],
            num_train_epochs=config["num_epochs"],
            lr_scheduler_type=config["scheduler"],
            warmup_ratio=config["warmup_ratio"],
            logging_steps=50, # Ray Tune中は少し多めに
            evaluation_strategy="epoch", # エポックごとに評価
            save_strategy="no", # Ray Tune中はモデルを保存しない (ディスクスペース節約)
            report_to="none", # Ray TuneがレポートするのでTrainerのレポートはオフ
            fp16=True, # A100ならTrue
            # auto_find_batch_size=False, # Ray Tuneでバッチサイズを探索するためFalse
        )

        from transformers import Trainer # Trainerをここでインポート
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_val,
            tokenizer=tokenizer,
        )

        train_result = trainer.train()
        eval_metrics = trainer.evaluate()

        # Ray Tuneに検証ロスを報告
        ray.train.report({"val_loss": eval_metrics.get("eval_loss", float('inf'))})


    # Ray Tuneの設定
    search_space = {
        "learning_rate": tune.loguniform(1e-5, 5e-4),
        "batch_size": tune.choice([1, 2]), # A100でも8Bモデルならバッチサイズは小さめから
        "gradient_accumulation_steps": tune.choice([4, 8, 16]),
        "num_epochs": tune.choice([1, 2]), # 時間がかかるので少なめに
        "scheduler": tune.choice(["cosine", "linear"]),
        "warmup_ratio": tune.uniform(0.01, 0.1),
        "lora_r": tune.choice([LORA_R, LORA_R*2]), # LoRAランクも試す場合
    }

    # ASHAスケジューラ (早期停止アルゴリズム)
    scheduler = ASHAScheduler(
        metric="val_loss",
        mode="min",
        max_t=2,  # 最大エポック数 (num_epochsの最大値と合わせる)
        grace_period=1, # 最低1エポックは実行
        reduction_factor=2
    )

    # Rayの初期化 (Colabではリソース制限に注意)
    if ray.is_initialized():
        ray.shutdown()
    ray.init(num_cpus=4, num_gpus=1 if device == "cuda" else 0, ignore_reinit_error=True) # Colab Pro A100を想定

    tuner = tune.Tuner(
        tune.with_resources(train_with_params_for_ray, {"cpu": 4, "gpu": 1 if device == "cuda" else 0}),
        param_space=search_space,
        tune_config=tune.TuneConfig(
            num_samples=5,  # 試行回数 (時間と相談)
            scheduler=scheduler,
        ),
        run_config=ray.train.RunConfig(
            name="rag_llama3_finetune",
            stop={"training_iteration": 2}, # 最大3エポック/イテレーションで停止 (num_epochsと合わせる)
        )
    )

    results = tuner.fit()

    best_result = results.get_best_result(metric="val_loss", mode="min")
    best_config = best_result.config

    print(f"最適なハイパーパラメータ: {best_config}")
    print(f"対応する検証ロス: {best_result.metrics['val_loss']}")

    ray.shutdown()
    return best_config

## 14.(オプション) モデルのロードとハイパーパラメータチューニングの実行と、最適パラメータでのトレーニング

これは、セル12で定義したハイパーパラメータチューニングを実行したい場合に限って使用してください。
チューニング後、得られた最適なパラメータでモデルをトレーニングします。

---
⚠️ このセルを実行する場合、セル12は実行しないでください（またはその逆）。多くの時間とリソースが必要です。

In [ ]:
# (オプション) ハイパーパラメータチューニングの実行と、最適パラメータでのトレーニング

# --- チューニングを実行するかどうかのフラグ ---

# Trueにするとチューニングを実行
DO_HYPERPARAMETER_TUNING = True #@param {type: "boolean"}

if DO_HYPERPARAMETER_TUNING:
    if 'train_data' not in globals() or train_data is None or \
       'val_data' not in globals() or val_data is None:
        print("エラー: train_data または val_data が定義されていません。セル9を先に実行してください。")
    else:
        print("ハイパーパラメータチューニングを開始します...")
        # tune_hyperparameters 関数は、Ray Tune内部でモデルロードを行う想定
        # そして、最適な *設定* (config) を返す
        best_params_config = tune_hyperparameters( # best_params -> best_params_config に変更
            base_model_name=BASE_MODEL,
            train_data=train_data,
            val_data=val_data,
            output_dir_root=OUTPUT_DIR + "_tune_runs", # チューニング試行の出力先
            device=DEVICE
        )


        if best_params_config: # best_params_config がNoneでないことを確認
            print(f"最適なハイパーパラメータ設定が見つかりました: {best_params_config}")
            print("最適なパラメータ設定でモデルをトレーニングします...")

            # --- ここでモデルとトークナイザーを再度ロード ---
            # LoRAのRもチューニング対象にした場合、best_params_configから取得
            current_lora_r = best_params_config.get("lora_r", LORA_R) # デフォルトはグローバルLORA_R
            # LoRA AlphaはRの2倍が一般的だが、チューニング対象にするならそれもconfigから取得
            current_lora_alpha = best_params_config.get("lora_alpha", current_lora_r * 2)
            # LoRA Dropoutもチューニング対象ならconfigから取得
            current_lora_dropout = best_params_config.get("lora_dropout", LORA_DROPOUT)

            print(f"最適な設定でモデルをロードします: LORA_R={current_lora_r}, LORA_ALPHA={current_lora_alpha}, LORA_DROPOUT={current_lora_dropout}")
            model_for_final_train, tokenizer_for_final_train = load_model_for_training(
                base_model_name=BASE_MODEL,
                lora_r=current_lora_r,
                lora_alpha=current_lora_alpha,
                lora_dropout=current_lora_dropout
            )
            # --- モデルロードここまで ---

            if model_for_final_train and tokenizer_for_final_train:
                # 最適なパラメータでトレーニングを実行
                trained_model, trained_tokenizer = train_model(
                    model_to_train=model_for_final_train,
                    tokenizer_for_model=tokenizer_for_final_train,
                    training_dataset=train_data,
                    validation_dataset=val_data,
                    save_path_root=OUTPUT_DIR,
                    train_batch_size=best_params_config["batch_size"], # best_params_configから取得,
                    gradient_accumulation_steps=best_params_config["gradient_accumulation_steps"],
                    max_epochs=best_params_config["num_epochs"],
                    base_learning_rate=best_params_config["learning_rate"],
                    lr_scheduler_name=best_params_config["scheduler"],
                    warmup_ratio_of_total_steps=best_params_config["warmup_ratio"],
                    # logging_interval_stepsなどはbest_params_configに含まれていればそれを使う
                    # なければtrain_modelのデフォルト値を使用
                    logging_interval_steps=best_params_config.get("logging_interval_steps", 10),
                    evaluation_interval_steps=best_params_config.get("evaluation_interval_steps", 100),
                    checkpoint_save_interval_steps=best_params_config.get("checkpoint_save_interval_steps", 100),
                    compute_device=DEVICE
                )
                print(f"最適化されたパラメータでのトレーニングが完了し、モデルは {OUTPUT_DIR} に保存されました。")
            else:
                print("モデルのロードに失敗したため、トレーニングを実行できませんでした。")
        else:
            print("ハイパーパラメータチューニングで最適なパラメータが見つかりませんでした。")
else:
    print("ハイパーパラメータチューニングはスキップされました (DO_HYPERPARAMETER_TUNING=False)。")
    print("セル13でデフォルトパラメータによるトレーニングを実行したか確認してください。")

## 15.RAG推論パイプライン作成

ここでは、SFT済みモデル（LoRAアダプタをロード）とRAGシステム（FAISSベクトルストア）を組み合わせて、LangChainの推論パイプラインを作成する関数 create_rag_pipeline を定義しています。

### RAG・Langchain関連の補足
*   `HuggingFaceEmbeddings`: 推論時にもFAISSインデックスをロードするために埋め込みモデルを初期化します。
*   `FAISS.load_local()`: 保存されたFAISSインデックスを読み込みます。
*   `vectorstore.as_retriever()`: 読み込んだFAISSベクトルストアからリトリーバーを取得します。
*   `HuggingFacePipeline`: ファインチューニング済みのHugging FaceモデルパイプラインをLangChainのLLMとしてラップします。
*   `PromptTemplate`: RAG推論用のプロンプトテンプレート（コンテキスト、質問、回答の指示を含む）を定義します。
*   `RetrievalQA.from_chain_type()`: リトリーバー、LLM、プロンプトテンプレートを組み合わせて、質問応答のためのRAGチェーン (`qa_chain`) を構築します。

In [21]:
# RAG推論パイプライン作成関数 (create_rag_pipeline)
def create_rag_pipeline(base_model_name, finetuned_model_path, embedding_model_name, faiss_index_path, device):
    """
    ファインチューニング済みモデルとRAGを組み合わせた推論パイプラインを作成します。
    """
    print("RAG推論パイプラインを作成中...")

    # 1. ベクトルストアの読み込み
    print(f"埋め込みモデル {embedding_model_name} をロード中...")
    try:
        embeddings = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            model_kwargs={"device": device}
        )
        if not os.path.exists(faiss_index_path):
            print(f"エラー: FAISSインデックス '{faiss_index_path}' が見つかりません。セル7を再実行してください。")
            return None

        print(f"FAISSインデックス {faiss_index_path} をロード中...")
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True) # 最新版FAISS対応
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # 検索するドキュメント数
        print("ベクトルストアのロード完了。")
    except Exception as e:
        print(f"ベクトルストアまたは埋め込みモデルのロード中にエラー: {e}")
        return None

    # 2. モデルとトークナイザーのロード (ファインチューニング済み)
    print(f"ファインチューニング済みモデル {finetuned_model_path} をロード中...")
    bnb_config = get_bnb_config() # 4ビット量子化設定

    try:
        # まずベースモデルを量子化してロード
        base_llm_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True
        )

        # 次にLoRAアダプタをロードしてマージ
        # finetuned_model_path はLoRAアダプタが保存されているディレクトリを指す
        if not os.path.exists(os.path.join(finetuned_model_path, "adapter_config.json")):
             print(f"警告: {finetuned_model_path} にLoRAアダプタ設定ファイル (adapter_config.json) が見つかりません。")
             print(f"ベースモデル {base_model_name} をそのまま使用します。ファインチューニングが適用されない可能性があります。")
             model_for_inference = base_llm_model # フォールバック
        else:
            print(f"LoRAアダプタを {finetuned_model_path} からロード中...")
            model_for_inference = PeftModel.from_pretrained(base_llm_model, finetuned_model_path)
            # 推論時にはマージして高速化も可能だが、ここではアダプタをアタッチしたまま使う
            # model_for_inference = model_for_inference.merge_and_unload() # マージする場合

        # トークナイザーはLoRAアダプタと一緒に保存されたもの、またはベースモデルのものを使用
        # LoRAアダプタ保存時にtokenizerも保存されていればそちらを優先
        if os.path.exists(os.path.join(finetuned_model_path, "tokenizer_config.json")):
            tokenizer_path_for_inference = finetuned_model_path
        else:
            tokenizer_path_for_inference = base_model_name
            print(f"警告: {finetuned_model_path} にトークナイザー設定が見つかりません。ベースモデル {base_model_name} のトークナイザーを使用します。")

        tokenizer_for_inference = AutoTokenizer.from_pretrained(
            tokenizer_path_for_inference,
            trust_remote_code=True
        )
        if tokenizer_for_inference.pad_token is None: # pad_tokenがない場合
            tokenizer_for_inference.pad_token = tokenizer_for_inference.eos_token

        print("推論用モデルとトークナイザーのロード完了。")

    except Exception as e:
        print(f"推論用モデルのロード中にエラー: {e}")
        return None

    # 3. HuggingFacePipelineの作成
    pipe = pipeline(
        "text-generation",
        model=model_for_inference,
        tokenizer=tokenizer_for_inference,
        max_new_tokens=512,  # 生成する最大トークン数 (入力長ではない)
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15,
        do_sample=True, # サンプリングを有効にする
        # device=0 if device == "cuda" else -1 # pipelineにdevice指定 (device_map="auto"なら不要かも)
    )
    llm = HuggingFacePipeline(pipeline=pipe)

    # 4. RAGプロンプトテンプレート
    template = """### 指示:
以下のコンテキスト情報を使用して、質問に対する回答を生成してください。
コンテキスト情報に含まれる事実のみを使用し、含まれていない情報は推測しないでください。コンテキストに情報がない場合は、その旨を伝えてください。

### コンテキスト:
{context}

### 質問:
{question}

### 回答:
"""
    PROMPT = PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

    # 5. RAG QAチェーンの作成
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", # すべての取得コンテキストをプロンプトに含める
        retriever=retriever,
        return_source_documents=True, # 参照元ドキュメントも返す
        chain_type_kwargs={"prompt": PROMPT}
    )

    print("RAG推論パイプラインの作成が完了しました。")
    return qa_chain

## 16.Gradio（UI表示）のための推論関数とチャットボットインターフェース

ここで、Gradio UIから呼び出される推論関数とGradioのチャットボットインターフェースを定義し、UIを起動します。

---
⚠️ このセルを実行する前に、セル13またはセル14でモデルのファインチューニングが完了し、OUTPUT_DIR にモデルが保存されていることを確認してください。

⚠️ またセル7で FAISS_INDEX_PATH にベクトルストアが作成されていることを確認してください。

---
### RAG・Langchain関連の補足
*   `global_qa_chain({"query": message})`: セル15で作成されたLangChainの `RetrievalQA` チェーン (`global_qa_chain`) を使って、ユーザーの質問 (`message`) に対する回答生成と参照ドキュメントの取得を行います。

In [38]:
# Gradio UIのための推論関数とインターフェース (gr.Interface版)

# --- グローバル変数としてQAチェーンを保持 ---
global_qa_chain = None

def initialize_qa_chain_for_gradio():
    """Gradio UIのためにQAチェーンを初期化 (または再初期化) します。"""
    global global_qa_chain

    best_performing_model_dir = os.path.join(OUTPUT_DIR, "best_performing_model")
    final_trained_model_dir = os.path.join(OUTPUT_DIR, "final_trained_model")
    output_dir_direct = OUTPUT_DIR
    finetuned_model_dir_to_load = None

    if os.path.exists(os.path.join(best_performing_model_dir, "adapter_config.json")):
        finetuned_model_dir_to_load = best_performing_model_dir
        print(f"ファインチューニング済みモデルとして '{best_performing_model_dir}' を使用します。")
    elif os.path.exists(os.path.join(final_trained_model_dir, "adapter_config.json")):
        finetuned_model_dir_to_load = final_trained_model_dir
        print(f"ファインチューニング済みモデルとして '{final_trained_model_dir}' を使用します。")
    elif os.path.exists(os.path.join(output_dir_direct, "adapter_config.json")):
        finetuned_model_dir_to_load = output_dir_direct
        print(f"ファインチューニング済みモデルとしてルートディレクトリ '{output_dir_direct}' を使用します。")
    else:
        print(f"エラー: ファインチューニング済みモデルが以下のいずれのパスにも見つかりません:")
        print(f"  - {best_performing_model_dir}")
        print(f"  - {final_trained_model_dir}")
        print(f"  - {output_dir_direct} (直下)")
        print("セル13またはセル14でトレーニングを完了し、モデルが正しく保存されているか確認してください。")
        return False

    if not os.path.exists(FAISS_INDEX_PATH):
        print(f"エラー: FAISSインデックスが {FAISS_INDEX_PATH} に見つかりません。セル7を完了してください。")
        return False

    global_qa_chain = create_rag_pipeline(
        base_model_name=BASE_MODEL,
        finetuned_model_path=finetuned_model_dir_to_load,
        embedding_model_name=EMBEDDING_MODEL,
        faiss_index_path=FAISS_INDEX_PATH,
        device=DEVICE
    )
    if global_qa_chain:
        print("QAチェーンの初期化が完了しました。Gradio UIを起動できます。")
        return True
    else:
        print("QAチェーンの初期化に失敗しました。")
        return False

# GradioのInterfaceから呼び出される関数 (履歴なし版)
def rag_response_for_interface(user_question):
    """
    ユーザーの質問を受け取り、RAGパイプラインで応答を生成します。
    (gr.Interface用、履歴なし)
    """
    if global_qa_chain is None:
        return "エラー: QAチェーンが初期化されていません。Gradio UIを再起動するか、初期化処理を確認してください。"

    print(f"\nユーザーからの質問: {user_question}")
    try:
        response_dict = global_qa_chain({"query": user_question})
        answer = response_dict.get("result", "回答を生成できませんでした。")

        source_docs_info = "\n\n--- 参照ドキュメント ---\n"
        if response_dict.get("source_documents"):
            for i, doc in enumerate(response_dict["source_documents"]):
                source_name = doc.metadata.get('source', '不明なソース')
                # page_contentが非常に長い場合があるので、先頭部分だけ表示
                content_preview = doc.page_content[:200].replace('\n', ' ') + "..."
                source_docs_info += f"{i+1}. ソース: {source_name}\n   内容抜粋: {content_preview}\n"
        else:
            source_docs_info += "参照ドキュメントはありませんでした。\n"

        full_answer_with_sources = answer + source_docs_info # 回答に参照情報を付加

        print(f"LLMからの回答: {answer}")
        if response_dict.get("source_documents"):
             print(source_docs_info)
        return full_answer_with_sources
    except Exception as e:
        print(f"推論中にエラーが発生しました: {e}")
        return f"推論エラー: {e}"

# --- Gradio UIの構築 (gr.Interfaceを使用) ---
initialization_successful = initialize_qa_chain_for_gradio()

if initialization_successful:
    print("Gradioインターフェース (gr.Interface版) を起動します...")

    # Gradio UIのテーマ設定 (オプション)
    # theme = gr.themes.Soft() # または他のテーマ gr.themes.Default() など

    iface = gr.Interface(
        fn=rag_response_for_interface, # 上で定義した応答関数
        inputs=gr.Textbox(lines=3, placeholder="ここに質問を入力してください..."),
        outputs=gr.Textbox(label="回答 (参照ドキュメント情報を含む)", lines=15), # 出力行数を調整
        title="RAG Llama3 Q&A (1問1答)",
        description=(
            f"ファインチューニングされた `{BASE_MODEL}` と、`{DOCS_PATH}` の情報に基づいて回答します。\n"
            "このUIは1問1答形式で、会話の履歴は保持しません。"
        ),
        # theme=theme, # テーマを適用する場合
        allow_flagging="never" # 簡単のためフラグ機能をオフにする
    )

    # ColabでGradioを起動
    iface.launch(debug=True) # debug=Trueでエラー詳細表示
else:
    print("QAチェーンの初期化に失敗したため、Gradio UIを起動できません。")
    print("上記のエラーメッセージを確認し、必要な手順 (ドキュメント処理、トレーニング等) を実行してください。")

ファインチューニング済みモデルとして '/content/finetuned_model/final_trained_model' を使用します。
RAG推論パイプラインを作成中...
埋め込みモデル intfloat/multilingual-e5-large をロード中...


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--intfloat--multilingual-e5-large/snapshots/0dc5580a448e4284468b8909bae50fa925907bc5/config.json
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--intfloat--multilingual-e5-large

FAISSインデックス faiss_index をロード中...
ベクトルストアのロード完了。
ファインチューニング済みモデル /content/finetuned_model/final_trained_model をロード中...


target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

LoRAアダプタを /content/finetuned_model/final_trained_model からロード中...


loading file tokenizer.json
loading file tokenizer.model
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCau

推論用モデルとトークナイザーのロード完了。
RAG推論パイプラインの作成が完了しました。
QAチェーンの初期化が完了しました。Gradio UIを起動できます。
Gradioインターフェース (gr.Interface版) を起動します...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://44f80b7a4c11f86b57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-38-33aceaed53e2>:61: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_dict = global_qa_chain({"query": user_question})
Disabling tokenizer parallelism, we're using DataLoader multithreading already



ユーザーからの質問: 強化学習におけるマルコフ決定過程、方策、状態価値関数、行動価値関数について教えてください。
LLMからの回答: ### 指示:
以下のコンテキスト情報を使用して、質問に対する回答を生成してください。
コンテキスト情報に含まれる事実のみを使用し、含まれていない情報は推測しないでください。コンテキストに情報がない場合は、その旨を伝えてください。

### コンテキスト:
74 CHAPTER 3. FINITE MARKOV DECISION PROCESSES
Q*(s, driver)
Vputt
s a n d
green
!1
s a
n
d
!2!2
!3
!4
!1
!5
!6
!4
!3
!3 !2
!4
s a n d
green
!1
s a
n
d
!2
!3
!2
0
0
!"
!"
vputt
q*(s,driver)
Figure 3.6: A golf example: the state-value function for putting (above) and
the optimal action-value function for using the driver (below).

because it has never received any relevant sensations. In short, we don’t fault
an agent for not knowing something that matters, but only for having known
something and then forgotten it!
What we would like, ideally, is a state signal that summarizes past sensa-
tions compactly, yet in such a way that all relevant information is retained.
This normally requires more than the immediate sensations, but never more
than the complete history of all past sensations. A stat

## 17.Hugging Face Hubへのアップロード(任意)


## 18.ローカル環境にモデルをダウンロード（任意）

In [43]:
import os
import shutil
from google.colab import files

# ダウンロード対象のディレクトリパス
source_directory_path = "/content/finetuned_model/final_trained_model"

# 出力されるzipファイルの名前
output_zip_filename = "final_trained_model_adapter.zip"
# zipファイルを一時的に保存するパス
output_zip_path = f"/content/{output_zip_filename}"

# ディレクトリが存在するか確認
if os.path.exists(source_directory_path) and os.path.isdir(source_directory_path):
    print(f"ディレクトリ '{source_directory_path}' を圧縮中...")
    try:
        # shutil.make_archive(base_name, format, root_dir, base_dir)
        # base_name: 出力ファイル名 (拡張子なし)
        # format: 'zip', 'tar', 'gztar', 'bztar', or 'xztar'
        # root_dir: アーカイブするファイルのルートディレクトリ (このディレクトリからの相対パスでアーカイブされる)
        # base_dir: アーカイブするディレクトリ名 (root_dirからの相対パス)
        # ここでは、source_directory_path の親ディレクトリを root_dir にし、
        # source_directory_path の最後のディレクトリ名を base_dir にする

        archive_base_name = "/content/final_trained_model_adapter" # .zip は自動で付与される
        root_dir_for_archive = os.path.dirname(source_directory_path) # /content/finetuned_model
        base_dir_to_archive = os.path.basename(source_directory_path) # final_trained_model

        shutil.make_archive(
            base_name=archive_base_name,
            format='zip',
            root_dir=root_dir_for_archive,
            base_dir=base_dir_to_archive
        )

        # 正しいzipファイルパスを再確認 (shutil.make_archiveは拡張子を自動でつける)
        actual_output_zip_path = archive_base_name + ".zip" # これが /content/final_trained_model_adapter.zip になる

        if os.path.exists(actual_output_zip_path):
            print(f"圧縮ファイル '{actual_output_zip_path}' が作成されました。")
            print("ダウンロードを開始します...")
            files.download(actual_output_zip_path)
            print(f"'{actual_output_zip_path}' のダウンロードがトリガーされました。ブラウザのダウンロードを確認してください。")
        else:
            print(f"エラー: 圧縮ファイル '{actual_output_zip_path}' が見つかりません。")

    except Exception as e:
        print(f"圧縮またはダウンロード中にエラーが発生しました: {e}")
else:
    print(f"エラー: 指定されたディレクトリ '{source_directory_path}' が見つからないか、ディレクトリではありません。")

ディレクトリ '/content/finetuned_model/final_trained_model' を圧縮中...
圧縮ファイル '/content/final_trained_model_adapter.zip' が作成されました。
ダウンロードを開始します...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'/content/final_trained_model_adapter.zip' のダウンロードがトリガーされました。ブラウザのダウンロードを確認してください。
